In [55]:
#imports
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from random import random
from random import seed
from math import exp

In [56]:
# No. of data points
n_samples = 200

# No. of features (dimensions of the data)
n_features = 4

# No. of redundent features (linear combinations of other features)
n_redundant = 1

# No. of classes
n_classes = 2


In [57]:
X, y = make_classification(n_samples=n_samples, n_features=n_features,
                                            n_redundant=n_redundant, n_classes=n_classes)
df = pd.DataFrame(X, columns=['feature1', 'feature2', 'feature3', 'feature4'])
df['label'] = y
df.head()

,feature1,feature2,feature3,feature4,label
0,-0.656134,0.310525,0.936812,0.543369,1
1,-1.226402,-1.667919,1.468561,-0.191813,0
2,-0.810019,0.688634,1.194878,-0.438608,1
3,0.146412,-0.089074,-0.211529,-0.297711,1
4,-0.736674,-1.371241,0.835730,0.711622,0


In [58]:
df.to_csv("dataset1.csv")

In [59]:
#reading the dataset 
df=pd.read_csv('dataset1.csv',index_col=0)
df.head()

,feature1,feature2,feature3,feature4,label
0,-0.656134,0.310525,0.936812,0.543369,1
1,-1.226402,-1.667919,1.468561,-0.191813,0
2,-0.810019,0.688634,1.194878,-0.438608,1
3,0.146412,-0.089074,-0.211529,-0.297711,1
4,-0.736674,-1.371241,0.835730,0.711622,0


In [60]:
#initializing the network for weights
def initialize_network(n_inputs, n_hidden, n_outputs):
    network=list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [61]:
#activation function
def activate(weights, inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
        activation+=weights[i]*inputs[i]
    return activation

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))


In [62]:
#forward propogation
def forward_propagate(network,raw):
    inputs=raw
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activation=activate(neuron['weights'], inputs)
            neuron['output']=transfer(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs


def transfer_derivative(output):
    return output * (1.0 - output)

In [63]:
#backward propogation to learn
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [64]:
#update weights on training
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs=row[:-1]
        if i!=0:
            inputs=[neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]+=l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1]+=l_rate*neuron['delta']

In [65]:
#training the network
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [66]:
#predicting function
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [67]:
dataset=np.array(df[:])
dataset

array([[-6.56134419e-01,  3.10525422e-01,  9.36811952e-01,
         5.43368517e-01,  1.00000000e+00],
       [-1.22640243e+00, -1.66791931e+00,  1.46856143e+00,
        -1.91812638e-01,  0.00000000e+00],
       [-8.10018954e-01,  6.88634460e-01,  1.19487769e+00,
        -4.38607737e-01,  1.00000000e+00],
       [ 1.46412313e-01, -8.90743943e-02, -2.11529102e-01,
        -2.97710878e-01,  1.00000000e+00],
       [-7.36673787e-01, -1.37124111e+00,  8.35730071e-01,
         7.11621637e-01,  0.00000000e+00],
       [-8.59332781e-01,  1.99248882e+00,  1.42616166e+00,
        -4.13147185e-01,  1.00000000e+00],
       [-3.83900652e-01,  1.62377194e+00,  7.29296923e-01,
         6.08694716e-01,  1.00000000e+00],
       [-5.19391474e-01,  1.57499439e+00,  9.08563224e-01,
        -7.57117509e-02,  1.00000000e+00],
       [-1.27461570e+00, -1.36649422e+00,  1.57240139e+00,
         7.02798302e-02,  0.00000000e+00],
       [-1.54174510e+00, -1.43307716e+00,  1.92955402e+00,
         1.19139270e+00

In [68]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
print(n_inputs,n_outputs)

4 2


In [69]:
#splitting into test and train datset
train_dataset=dataset[:150]
test_dataset=dataset[150:]

In [70]:
#feeding the datset into the network
network=initialize_network(n_inputs,1,n_outputs)
train_network(network, train_dataset, 0.5, 100, n_outputs)

>epoch=0, lrate=0.500, error=67.556
>epoch=1, lrate=0.500, error=38.709
>epoch=2, lrate=0.500, error=27.563
>epoch=3, lrate=0.500, error=23.651
>epoch=4, lrate=0.500, error=21.861
>epoch=5, lrate=0.500, error=20.889
>epoch=6, lrate=0.500, error=20.288
>epoch=7, lrate=0.500, error=19.873
>epoch=8, lrate=0.500, error=19.558
>epoch=9, lrate=0.500, error=19.302
>epoch=10, lrate=0.500, error=19.086
>epoch=11, lrate=0.500, error=18.902
>epoch=12, lrate=0.500, error=18.743
>epoch=13, lrate=0.500, error=18.607
>epoch=14, lrate=0.500, error=18.490
>epoch=15, lrate=0.500, error=18.388
>epoch=16, lrate=0.500, error=18.300
>epoch=17, lrate=0.500, error=18.223
>epoch=18, lrate=0.500, error=18.156
>epoch=19, lrate=0.500, error=18.097
>epoch=20, lrate=0.500, error=18.045
>epoch=21, lrate=0.500, error=17.999
>epoch=22, lrate=0.500, error=17.958
>epoch=23, lrate=0.500, error=17.922
>epoch=24, lrate=0.500, error=17.889
>epoch=25, lrate=0.500, error=17.859
>epoch=26, lrate=0.500, error=17.832
>epoch=27, 

In [71]:
#learned weights of the network
for layer in network:
    print(layer)

[{'weights': [1.337920763536887, 9.213781919823663, 0.8977625016455786, -0.7476764984890443, 2.8825284014801036], 'output': 0.999896065659801, 'delta': 8.261176901039936e-06}]
[{'weights': [-6.481174919546664, 4.055439882951116], 'output': 0.0817407835609989, 'delta': -0.0061354000990732575}, {'weights': [6.481027755942583, -4.05532362808749], 'output': 0.9182568723353219, 'delta': 0.006135736334185702}]


In [72]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [73]:
#applying on training dataset
y_train=[]
pred=[]
for row in train_dataset:
    prediction = predict(network, row)
    y_train.append(int(row[-1]))
    pred.append(prediction)

In [74]:
print("Accuracy: ",accuracy_score(y_train,pred))
print("Confusion Matrix: ",confusion_matrix(y_train,pred))
print("Precision: ",precision_score(y_train, pred))
print("recall: ",recall_score(y_train, pred))

Accuracy:  0.94
Confusion Matrix:  [[69  8]
 [ 1 72]]
Precision:  0.9
recall:  0.9863013698630136


In [75]:
#applying on testing dataset
y_test=[]
pred=[]
for row in test_dataset:
    prediction = predict(network, row)
    y_test.append(row[-1])
    pred.append(prediction)

In [76]:
print("Accuracy: ",accuracy_score(y_test,pred))
print("Confusion Matrix: ",confusion_matrix(y_test,pred))
print("Precision: ",precision_score(y_test, pred))
print("recall: ",recall_score(y_test, pred))

Accuracy:  0.9
Confusion Matrix:  [[19  3]
 [ 2 26]]
Precision:  0.896551724137931
recall:  0.9285714285714286
